In [21]:
import pandas as pd

In [20]:
data = pd.read_csv("../data/foodb/Content.csv")

C:\Users\oszud\AppData\Local\Temp\ipykernel_17024\2274376406.py:1: DtypeWarning: Columns (4,5,6,7,8,9,13,14,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/foodb/Content.csv")


Original unit of protein mg/100g

In [23]:
data_prot = data[data.orig_source_name == "Protein"]
data_prot.head(3)

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
399,400,2,Nutrient,4,9148,"Kiwifruit, green, raw",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,1140.0,raw,0
400,401,2,Nutrient,4,9445,"Kiwifruit, gold, raw",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,1230.0,raw,0
410,411,2,Nutrient,5,2026,"Spices, onion powder",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,10410.0,dried or powder,0


In [24]:
print("Raw protein info: ", len(data_prot))

Raw protein info:  7336


In [5]:
data_n = data[data.orig_source_name == "Protein, total-N"]
data_n.head(3)

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
396,397,2,Nutrient,2,674,"Cabbage, savoy, raw",NaN,NaN,0002,"Protein, total-N",...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,300.0,raw,0
398,399,2,Nutrient,4,465,"Kiwi fruit, raw",NaN,NaN,0002,"Protein, total-N",...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,200.0,raw,0
403,404,2,Nutrient,5,148,"Onion, raw",NaN,NaN,0002,"Protein, total-N",...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,300.0,raw,0


As you see - there is also "Protein, total-N" type of records. Those records tell us about nitrogen content. Internet says that sometimes it is used to estimate how much protein is actually in a food. However, because it is not that straightforward - I suggest we do not dig into it right now. If we won't obtain enough info - it might be somehow useful.

Based on 
```
 from ydata_profiling import ProfileReport

 profile = ProfileReport(data_prot, title='Protein FooDB Report')
 profile
```
and
```
 data_prot.COLNAME.unique()
```
we'll drop some columns

In [6]:
data_prot = data[data.orig_source_name == "Protein"]
data_prot = data_prot.drop(columns=['creator_id', 'updater_id', 'created_at', 'updated_at', 
                                    'orig_citation', 'citation', 'citation_type',
                                    'orig_unit', 'source_id', 'orig_source_name', 'orig_source_id',
                                    'orig_food_scientific_name', 'orig_food_part', 'orig_min', 'orig_max',
                                    'orig_method', 'orig_unit_expression', 'source_type'])

In [7]:
data_prot.head(3)

,id,food_id,orig_food_id,orig_food_common_name,orig_content,standard_content,preparation_type,export
399,400,4,9148,"Kiwifruit, green, raw",1140.0,1140.0,raw,0
400,401,4,9445,"Kiwifruit, gold, raw",1230.0,1230.0,raw,0
410,411,5,2026,"Spices, onion powder",10410.0,10410.0,dried or powder,0


In [8]:
data_prot[data_prot['orig_food_common_name'].str.contains('Onions')][:5]

,id,food_id,orig_food_id,orig_food_common_name,orig_content,standard_content,preparation_type,export
411,412,5,11282,"Onions, raw",1100.0,1100.0,raw,0
412,413,5,11283,"Onions, cooked, boiled, drained, without salt",1360.0,1360.0,cooked,0
413,414,5,11284,"Onions, dehydrated flakes",8950.0,8950.0,dried or powder,0
414,415,5,11285,"Onions, canned, solids and liquids",850.0,850.0,NaN,0
415,416,5,11286,"Onions, yellow, sauteed",950.0,950.0,cooked,0


In [26]:
names = data.orig_source_name.unique()
prot_names = []
for name in names:
    n = str(name)
    if "Protein" in n:
        prot_names.append(name)
        
prot_names.remove('Protein, total-N')
prot_names

['Protein, total', 'Protein', 'Adjusted Protein']

In [27]:
foodb_ingr = set()

for prot_name in prot_names:
    data_prot = data[data.orig_source_name == prot_name]
    data_prot = data_prot.drop(columns=['creator_id', 'updater_id', 'created_at', 'updated_at', 
                                    'orig_citation', 'citation', 'citation_type',
                                    'orig_unit', 'source_id', 'orig_source_name', 'orig_source_id',
                                    'orig_food_scientific_name', 'orig_food_part', 'orig_min', 'orig_max',
                                    'orig_method', 'orig_unit_expression', 'source_type'])
    
    for record in data_prot['orig_food_common_name']:
        items = record.split(',')
        first_item = items[0].strip()
        foodb_ingr.add(first_item)

In [28]:
len(foodb_ingr)

1331

In [9]:
nlg = pd.read_csv("../data/recipeNLG/full_dataset.csv")

In [10]:
nlg.head(3)

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."


In [30]:
import ast

ingredients = set()

for record in nlg['NER']:
    item_list = ast.literal_eval(record)
    for item in item_list:
        ingredients.add(item)

In [31]:
len(ingredients)

234059

https://www.kaggle.com/datasets/thedevastator/now-with-more-nutrients/

In [29]:
ingrs = pd.read_csv("../data/cleaned_ingredients.csv")

In [30]:
ingrs.head(3)

,NDB_No,Descrip,Energy_kcal,Protein_g,Saturated_fats_g,Fat_g,Carb_g,Fiber_g,Sugar_g,Calcium_mg,...,VitC_mg,Thiamin_mg,Riboflavin_mg,Niacin_mg,VitB6_mg,Folate_mcg,VitB12_mcg,VitA_mcg,VitE_mg,VitD2_mcg
0,1001,butter with salt,717.0,0.85,51.368,81.11,0.06,0.0,0.06,24.0,...,0.0,0.005,0.034,0.042,0.003,3.0,0.17,684.0,2.32,0.0
1,1002,butter whipped w salt,718.0,0.49,45.390,78.30,2.87,0.0,0.06,23.0,...,0.0,0.007,0.064,0.022,0.008,4.0,0.07,683.0,1.37,0.0
2,1003,butter oil anhydrous,876.0,0.28,61.924,99.48,0.00,0.0,0.00,4.0,...,0.0,0.001,0.005,0.003,0.001,0.0,0.01,840.0,2.8,0.0


In [31]:
kaggle_ingrs = set() 

for record in ingrs['Descrip']:
    kaggle_ingrs.add(record)  

In [32]:
len(kaggle_ingrs)

9313